In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
!unzip -q "/content/drive/MyDrive/project/Dataset/jeans&trousersimagedatagenerator.zip"

replace base/testing_data/jeans/jeans (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [43]:
import os
import time
import numpy as np
import tensorflow as tf
from keras import backend as K
import matplotlib.pyplot as plt
from google.colab import drive
from keras.models import load_model
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**Set Path for Train,Test,Validation**

In [44]:
path = '/content/base'
train_dir = '/content/base/training_data'
validation_dir = '/content/base/validation_data'
test_dir = '/content/base/testing_data'

In [45]:
train_jeans_dir = '/content/base/training_data/jeans'
train_trousers_dir = '/content/base/training_data/trousers'

validation_jeans_dir = '/content/base/validation_data/jeans'
validation_trousers_dir ='/content/base/validation_data/trousers'

test_jeans_dir = '/content/base/testing_data/jeans'
test_trousers_dir = '/content/base/testing_data/trousers'

In [46]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [47]:
num_jeans_train = len(os.listdir(train_jeans_dir))
num_trousers_train = len(os.listdir(train_trousers_dir))

num_jeans_validation = len(os.listdir(validation_jeans_dir))
num_trousers_validation = len(os.listdir(validation_trousers_dir))

num_jeans_test = len(os.listdir(test_jeans_dir))
num_trousers_test = len(os.listdir(test_trousers_dir))

**Printing Total Numbers of Testing, Training and validation images**

In [48]:
print('Total Training Images of Jeanss',num_jeans_train)
print('Total Training Images of Trouserss',num_trousers_train)
print('\n************************\n')
print('Total Validation Images of Jeanss',num_jeans_validation)
print('Total Validation Imagges of Trouserss',num_trousers_validation)
print('\n************************\n')
print('Total Testing Images of Jeanss',num_jeans_test)
print('Total Testing Images of Trouserss',num_trousers_test)

Total Training Images of Jeanss 119
Total Training Images of Trouserss 120

************************

Total Validation Images of Jeanss 119
Total Validation Imagges of Trouserss 120

************************

Total Testing Images of Jeanss 119
Total Testing Images of Trouserss 120


In [49]:
total_train = num_jeans_train+num_trousers_train
total_validation = num_jeans_validation+num_trousers_validation
total_test = num_jeans_test+num_trousers_test

In [50]:
print('Total Training Images',total_train)
print('Total Validation Images',total_validation)
print('Total Testing Images',total_test)

Total Training Images 239
Total Validation Images 239
Total Testing Images 239


**Defining Batch-size and image_shape for deep learning model**

In [51]:
BATCH_SIZE = 120
IMG_SHAPE  = 150

**Function for plotting images**

In [52]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

**Applying Data Agumentation**

In [53]:
image_gen_train = ImageDataGenerator(rescale=1./255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

train_data_gen = image_gen_train.flow_from_directory(train_dir,
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     target_size=(img_width, img_height),
                                                     class_mode='binary')

Found 239 images belonging to 2 classes.


In [54]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

UnidentifiedImageError: ignored

**Pre-Processing for testing and validation data**

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=validation_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='binary')

image_gen_test = ImageDataGenerator(rescale=1./255)

test_data_gen = image_gen_test.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=test_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='binary')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Conv2D,Flatten,Dense,Activation,MaxPooling2D
from keras import backend as k

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())    

In [ ]:
import keras
import tensorflow as tf
tf.keras.utils.plot_model(model,to_file="my_model.png")

In [ ]:
from PIL import Image

In [ ]:
epochs=100
history = model.fit(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_validation / float(BATCH_SIZE))),
    verbose= 1
)

In [ ]:
import matplotlib.pyplot as plt
history_dict=history.history
loss_values=history_dict['loss']
val_loss_values=history_dict['val_loss']
epochs=range(1,len(loss_values)+1)
line1=plt.plot(epochs,val_loss_values,label='validation/test loss')
line2=plt.plot(epochs,loss_values,label='Training loss')
plt.setp(line1,linewidth=2.0,marker='+',markersize=10.0)
plt.setp(line2,linewidth=2.0,marker='4',markersize=10.0)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show

In [ ]:
import matplotlib.pyplot as plt
history_dict=history.history
acc_values=history_dict['accuracy']
val_acc_values=history_dict['val_accuracy']
epochs=range(1,len(loss_values)+1)
line1=plt.plot(epochs,val_acc_values,label='validation/Test Accuracy ')
line2=plt.plot(epochs,acc_values,label='Training Accuracy')
plt.setp(line1,linewidth=2.0,marker='+',markersize=10.0)
plt.setp(line2,linewidth=2.0,marker='4',markersize=10.0)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show

In [ ]:
history.history

In [ ]:
model.save('Jeans&Trousers.h5')
print('model save')